# Regex in Pandas

**Inhalt:** Regular Expressions in Pandas anwenden

**Nötige Skills:** Regex in Python

**Lernziele:**
- Ein praktisches Beispiel kennenlernen, wo Regex nützlich sein kann

# Das Beispiel

Das Bundesamt für Statistik stellt oft Dateien in verknorkster Form zur Verfügung, zum Beispiel, wenn man Auswertungen nach Gemeinden, Bezirken und Kantonen über das interaktive Portal generieren lässt: https://www.pxweb.bfs.admin.ch/pxweb/de/

Das vorliegende Beispiel beinhaltet die Bevölkerungszahlen, gegliedert nach Zivilstand einerseits (ledig, verheiratet, etc) und nach räumlicher Struktur (Gemeinden, Bezirke, Kantone) andererseits.

Das File ist gespeichert unter `dataprojects/BFS/px-x-0102010000_103.xlsx`

Unser 1. Ziel ist: Wir wollen die Tabelle filtern
- nur die Gemeinden
- nur die Bezirke
- nur die Kantone

Unser 2. Ziel ist: Wir wollen bei den Gemeinden die jeweilige Nummer isolieren

## Vorbereitung

In [10]:
import pandas as pd

In [11]:
import re

In [12]:
%matplotlib inline

## Datei laden

In [13]:
path = 'dataprojects/BFS/px-x-0102010000_103.xlsx'

In [14]:
df = pd.read_excel(path)

## Explorieren

In [15]:
df.head(20)

,Einheit,Zivilstand,Anzahl
0,Schweiz,Ledig,3650651
1,Schweiz,Verheiratet,3583008
2,Schweiz,Verwitwet,407408
3,Schweiz,Geschieden,685622
4,- Zürich,Ledig,666873
5,- Zürich,Verheiratet,610396
6,- Zürich,Verwitwet,63173
7,- Zürich,Geschieden,125889
8,>> Bezirk Affoltern,Ledig,21785
9,>> Bezirk Affoltern,Verheiratet,23865


Welche Einheitstypen gibt es? Und welches Muster haben sie?
- Gemeinde ("...... 9999 Gemeindename")
- Bezirk (">> Bezirsname")
- Kantone ("- Kantonsname")
- Land ("Land")

## Pandas-Funktionen, die Regex brauchen können

Einige Befehle heissen leicht anders, funktionieren aber sehr ähnlich wie in der re.Library

- **`str.contains(r"regex")`**: das Pendant zu `re.search()` - ja/nein-Antwort

- **`str.extract(r"regex")`**: auch ähnlich wie `re.search()` - Suchergebnis als Antwort

- **`str.replace(r"regex", "str")`**: das Pendant zu `re.sub()` - ersetzt Match mit String

Wir wenden diese Funktionen jetzt an.

### Aber zuerst ...

**Quiz:** Zuerst brauchen wir die Regex-Ausdrücke, um die Einheiten zu erkennen.

Am besten mit Tests beginnen, ob die Regex an einer Einheit anschlägt - und jeweils auch testen, ob die Regex bei Einheiten, die wir *nicht* wollen, auch *nicht* anschlägt.

**Test für Kantone**

In [16]:
# Unsere Regex
regex_k = r"^- "

In [17]:
# Schlägt die regex bei den Kantonen an?
# (Gut ist, wenn wir ein "Match"-Objekt erhalten)
re.search(regex_k, "- Aargau")

<re.Match object; span=(0, 2), match='- '>

In [18]:
# Schlägt die regex bei den Bezirken NICHT an?
# (Gut ist hier, wenn wir nichts zurückerhalten)
re.search(regex_k, ">> Wahlkreis Luzern-Stadt")

In [19]:
# Schlägt die regex bei den Gemeinden NICHT an?
re.search(regex_k, "......0001 Aeugst am Albis")

**Test für Bezirke**

In [20]:
# Unsere Regex
regex_b = r"^>> "

In [21]:
# Schlägt die regex bei den Bezirken an?
re.search(regex_b, ">> Wahlkreis Luzern-Stadt")

<re.Match object; span=(0, 3), match='>> '>

In [22]:
# Schlägt die regex bei den Kantonen NICHT an?
re.search(regex_b, "- Aargau")

In [23]:
# Schlägt die regex bei den Gemeinden NICHT an?
re.search(regex_b, "......0001 Aeugst am Albis")

**Test für Gemeinden**

In [24]:
# Unsere Regex
regex_g = r"^\.{6}"

In [25]:
# Test für Gemeinde
re.search(regex_g, "......0001 Aeugst am Albis")

<re.Match object; span=(0, 6), match='......'>

In [26]:
# Schlägt die regex bei den Kantonen NICHT an?
re.search(regex_g, "- Aargau")

In [27]:
# Schlägt die regex bei den Bezirken NICHT an?
re.search(regex_g, ">> Wahlkreis Luzern-Stadt")

zum Zusammenfassen:

In [28]:
regex_k = r"^- "
regex_b = r"^>> "
regex_g = r"^\.{6}"

## 1. Daten filtern

Nachdem wir nun die passenden Regex-Ausdrücke gefunden haben, können wir unsere Daten filtern.

Ziel: Wir erstellen ein separates DF für Gemeinden, Bezirke, Kantone.

Hier können wir die Funktion `str.contains()` gut brauchen.

Wir testen damit mal, ob die Einträge in der Spalte "Einheit" eine Gemeinde sind:

In [29]:
df['Einheit'].str.contains(regex_g).head(15)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12     True
13     True
14     True
Name: Einheit, dtype: bool

Basierend auf dieser True/False-Liste können wir nun die Tabelle filtern.

In [30]:
df[df['Einheit'].str.contains(regex_g)].head()

,Einheit,Zivilstand,Anzahl
12,......0001 Aeugst am Albis,Ledig,852
13,......0001 Aeugst am Albis,Verheiratet,908
14,......0001 Aeugst am Albis,Verwitwet,74
15,......0001 Aeugst am Albis,Geschieden,147
16,......0002 Affoltern am Albis,Ledig,5000


Wir speichern den Output in einer separaten DF (als Kopie, damit wir nachher keinen Ärger kriegen).

In [31]:
df_g = df[df['Einheit'].str.contains(regex_g)].copy()

**Quiz:** Erstellen Sie zwei weitere Dataframes, in denen nur die Kantone und Bezirke sind.

In [32]:
df_k = df[df['Einheit'].str.contains(regex_k)].copy()

In [33]:
df_b = df[df['Einheit'].str.contains(regex_b)].copy()

In [34]:
df_k

,Einheit,Zivilstand,Anzahl
4,- Zürich,Ledig,666873
5,- Zürich,Verheiratet,610396
6,- Zürich,Verwitwet,63173
7,- Zürich,Geschieden,125889
732,- Bern / Berne,Ledig,437186
733,- Bern / Berne,Verheiratet,439372
734,- Bern / Berne,Verwitwet,56513
735,- Bern / Berne,Geschieden,84364
2200,- Luzern,Ledig,181455
2201,- Luzern,Verheiratet,172144


## 2. Daten extrahieren

Wir wenden uns den Gemeinden zu. Unser Ziel: Die Gemeindenummer in eine separate Spalte ziehen.

In [36]:
df_g.head(2)

,Einheit,Zivilstand,Anzahl
12,......0001 Aeugst am Albis,Ledig,852
13,......0001 Aeugst am Albis,Verheiratet,908


Hier kommt die Funktion `str.extract()` gelegen. Um sie zu benutzen, müssen wir zwei Dinge wissen:
1. die Regex, die zu dem gesamten Ausdruck passt.
2. Den Teil der Regex, den wir in Klammern `()` setzen und extrahieren wollen.

In [50]:
    
df_g['Einheit'].str.extract(r"(\d{4})").head()

,0
12,0001
13,0001
14,0001
15,0001
16,0002


Wir können den Output dieser Funktion für die neue Spalte setzen.

In [51]:
df_g['Code'] = df_g['Einheit'].str.extract(r"(\d{4})")

In [52]:
df_g.head(10)

,Einheit,Zivilstand,Anzahl,Code
12,......0001 Aeugst am Albis,Ledig,852,0001
13,......0001 Aeugst am Albis,Verheiratet,908,0001
14,......0001 Aeugst am Albis,Verwitwet,74,0001
15,......0001 Aeugst am Albis,Geschieden,147,0001
16,......0002 Affoltern am Albis,Ledig,5000,0002
17,......0002 Affoltern am Albis,Verheiratet,5159,0002
18,......0002 Affoltern am Albis,Verwitwet,545,0002
19,......0002 Affoltern am Albis,Geschieden,1004,0002
20,......0003 Bonstetten,Ledig,2336,0003
21,......0003 Bonstetten,Verheiratet,2452,0003


**Quiz:** Erstellen Sie im Gemeinde-DF eine weitere Spalte ("Name"), die den Namen der Gemeinde enthält!

In [54]:
df_g['Name'] = df_g['Einheit'].str.extract(r"\.{6}\d{4} (.+)$")

In [55]:
df_g.head()

,Einheit,Zivilstand,Anzahl,Code,Name
12,......0001 Aeugst am Albis,Ledig,852,0001,Aeugst am Albis
13,......0001 Aeugst am Albis,Verheiratet,908,0001,Aeugst am Albis
14,......0001 Aeugst am Albis,Verwitwet,74,0001,Aeugst am Albis
15,......0001 Aeugst am Albis,Geschieden,147,0001,Aeugst am Albis
16,......0002 Affoltern am Albis,Ledig,5000,0002,Affoltern am Albis


In [56]:
df_g.pop('Einheit')

12         ......0001 Aeugst am Albis
13         ......0001 Aeugst am Albis
14         ......0001 Aeugst am Albis
15         ......0001 Aeugst am Albis
16      ......0002 Affoltern am Albis
17      ......0002 Affoltern am Albis
18      ......0002 Affoltern am Albis
19      ......0002 Affoltern am Albis
20              ......0003 Bonstetten
21              ......0003 Bonstetten
22              ......0003 Bonstetten
23              ......0003 Bonstetten
24         ......0004 Hausen am Albis
25         ......0004 Hausen am Albis
26         ......0004 Hausen am Albis
27         ......0004 Hausen am Albis
28                ......0005 Hedingen
29                ......0005 Hedingen
30                ......0005 Hedingen
31                ......0005 Hedingen
32         ......0006 Kappel am Albis
33         ......0006 Kappel am Albis
34         ......0006 Kappel am Albis
35         ......0006 Kappel am Albis
36                  ......0007 Knonau
37                  ......0007 Knonau
38          

In [57]:
df_g.head()

,Zivilstand,Anzahl,Code,Name
12,Ledig,852,0001,Aeugst am Albis
13,Verheiratet,908,0001,Aeugst am Albis
14,Verwitwet,74,0001,Aeugst am Albis
15,Geschieden,147,0001,Aeugst am Albis
16,Ledig,5000,0002,Affoltern am Albis


## Übung

### Teil 1

**Hier haben Sie Gelegenheit noch etwas mit den Regex-Funktionen in Pandas zu experimentieren.**

Wir arbeiten wieder mit dem Haupt-Dataframe am Anfang: `df`.

**Aufgabe 1:** Erstellen Sie im DF eine zusätzliche Spalte mit dem Namen "Typ". Darin soll stehen, ob es sich bei der betreffenden Zeile um eine Gemeinde, einen Bezirk, einen Kanton oder ein Land handelt.

Tipp: Arbeiten Sie mit `df.loc[]`

In [35]:

df_g

,Einheit,Zivilstand,Anzahl
12,......0001 Aeugst am Albis,Ledig,852
13,......0001 Aeugst am Albis,Verheiratet,908
14,......0001 Aeugst am Albis,Verwitwet,74
15,......0001 Aeugst am Albis,Geschieden,147
16,......0002 Affoltern am Albis,Ledig,5000
17,......0002 Affoltern am Albis,Verheiratet,5159
18,......0002 Affoltern am Albis,Verwitwet,545
19,......0002 Affoltern am Albis,Geschieden,1004
20,......0003 Bonstetten,Ledig,2336
21,......0003 Bonstetten,Verheiratet,2452


**Aufgabe 2:** Bereinigen Sie die Bezeichnungen in der Spalte "Einheit". Wir wollen keine Nummern und keine Sonderzeichen mehr darin sehen, nur noch die Namen der Gemeinden, Bezirke und Kantone.

Tipp: Arbeiten Sie mit `str.replace()`

In [ ]:
# zum Testen, ob es bei den Kantonen funktioniert hat:
df[df['Typ'] == 'Kanton'].head(10)

In [ ]:
# zum Testen, ob es bei den Bezirken funktioniert hat:
df[df['Typ'] == 'Bezirk'].head(10)

In [ ]:
# zum Testen, ob es bei den Gemeinden funktioniert hat:
df[df['Typ'] == 'Gemeinde'].head(10)

### Teil 2

**Hier können Sie nochmals etwas Pivoting und Plotting trainieren**

In [ ]:
df.to_csv('dataprojects/BFS/Einheiten_kategorisiert.csv', index=False)

Zur Sicherheit laden wir die Daten nochmals neu - mit dem Stand der Übung bis hier.

In [ ]:
df = pd.read_csv('dataprojects/BFS/Einheiten_kategorisiert.csv')

**Aufgabe 1:** Finden Sie die drei Gemeinden mit der höchsten Quote an Verwitweten Personen in der Bevölkerung.

Tipp zum Vorgehen:
1. DF nach Gemeinden filtern
1. DF nach Zivilstand pivotieren
1. Total der Bevölkerung mit `sum()` ausrechnen (`axis=1`)
1. Verwitweten-Quote ausrechnen
1. DF sortieren

In [ ]:
# DF nach gemeinden filtern und pivotieren


In [ ]:
# Bevölkerungstotal ausrechnen pro Gemeinde


In [ ]:
# Verwitweten-Quote ausrechnen


In [ ]:
# Sortieren und Top 3


**Aufgabe 2:** Welcher Kanton hat die höchste Ledigen-Quote?

In [ ]:
# DF nach Kantonen filtern und pivotieren


In [ ]:
# Bevölkerungstotal ausrechnen pro Kanton


In [ ]:
# Ledigen-Quote ausrechnen


In [ ]:
# Sortieren und obersten anzeigen


**Aufgabe 3:** Zeichnen Sie einen Bar-Chart mit der Verheirateten-Quote aller Kantone

Tipp: Benutzen Sie das pivotierte DF, das Sie in Aufgabe 2 erstellt haben.

In [ ]:
# Verheirateten-Quote ausrechnen


In [ ]:
#Sortieren und plotten


**Aufgabe 4:** Zeichnen Sie ein Stacked-Bar-Chart mit den relativen Zivilstands-Anteilen in zwei Gemeinden:
- Corippo
- Meienried

Tipp zu Vorgehen:
1. DF nach Gemeinden filtern und pivotieren
1. Total der Einwohner für jede Gemeinde (in separater Series ablegen)
1. Pivotiertes DF mit `div()` durch das Total dividieren (`axis=1`)
1. mit `.loc[]` die beiden Gemeinden wählen
1. Plot mit `stacked=True`

In [ ]:
# DF nach gemeinden filtern und pivotieren


In [ ]:
# Total der Einwohner für jede Gemeinde


In [ ]:
# Neues DF mit relativen Werten


In [ ]:
# Nur Meienried und Corippo anzeigen


In [ ]:
# Plot
